#### 03 – Analytics & Gold Layer Transformation

**Goal:**
Execute analytical queries to answer the final coursework questions and materialize the results for reporting.
>* **Analysis:** Calculate total revenue, top zones, and trip metrics.  
>* **Validation:** Ensure data consistency across the joined tables.  
>* **Gold Layer:** Save the final insights into a permanent table.

---

#### Setup & Auth

In [ ]:
import os
import pandas as pd
from google.cloud import bigquery
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# find project root 
root_path = Path.cwd().parent if 'notebooks' in os.getcwd() else Path.cwd()
load_dotenv(dotenv_path=root_path / ".env")

project_id = os.getenv('GCP_PROJECT_ID')
bq_client = bigquery.Client(project=project_id)

# define table references
FACT_TABLE = f"{project_id}.trips_data_silver.ext_green_taxi"
ZONES_TABLE = f"{project_id}.trips_data_silver.zones"
GOLD_TABLE = f"{project_id}.trips_data_gold.fact_daily_revenue"

#### Validation (Data Integrity Check)

In [ ]:
# check for orphaned LocationIDs
validation_query = f"""
    SELECT COUNT(DISTINCT t.PULocationID) as orphaned_ids
    FROM `{FACT_TABLE}` t
    LEFT JOIN `{ZONES_TABLE}` z ON t.PULocationID = z.LocationID
    WHERE z.LocationID IS NULL
"""
orphans = bq_client.query(validation_query).to_dataframe().iloc[0]['orphaned_ids']

if orphans == 0:
    print("Validation Passed: All Pickup IDs match the Zone Lookup.")
else:
    print(f"Validation Warning: {orphans} Pickup IDs do not have a matching Zone.")

#### Analytics

**Q1 Short Trip Volume** 

Count trips in November 2025 with a distance $\le$ 1 mile.

In [ ]:
coursework_q1 = f"""
    SELECT COUNT(*) AS short_trips
    FROM `{FACT_TABLE}`
    WHERE lpep_pickup_datetime >= '2025-11-01'
      AND lpep_pickup_datetime <  '2025-12-01'
      AND trip_distance <= 1;
"""

df_q1 = bq_client.query(coursework_q1).to_dataframe()
retval = df_q1['short_trips'].iloc[0]
print(f"Result: {retval}")

**Q2 Longest Trip Day** 

Identify the pickup day with the longest trip distance in November 2025 (filtering out trips $\ge$ 100 miles as potential data errors).

In [ ]:
coursework_q2 = f"""
    SELECT 
        DATE(lpep_pickup_datetime) AS pickup_day, 
        trip_distance
    FROM `{FACT_TABLE}`
    WHERE lpep_pickup_datetime >= '2025-11-01'
      AND lpep_pickup_datetime <  '2025-12-01'
      AND trip_distance < 100
    ORDER BY trip_distance DESC
    LIMIT 1
"""

df_q2 = bq_client.query(coursework_q2).to_dataframe()

trip_distance = df_q2['trip_distance'].iloc[0]
pickup_day = df_q2['pickup_day'].iloc[0]
print(f"Result: The longest valid trip was {trip_distance} miles on `{pickup_day}`")

**Q3 Highest Revenue: Pickup Zone** 

Identify the zone with the largest `total_amount` sum for trips starting on November 18th, 2025.

In [ ]:
coursework_q3 = f"""
    SELECT
        z.Zone AS pickup_zone,
        SUM(t.total_amount) AS total_revenue
    FROM `{FACT_TABLE}` t
    LEFT JOIN `{ZONES_TABLE}` z
      ON t.PULocationID = z.LocationID
    WHERE DATE(t.lpep_pickup_datetime) = '2025-11-18'
    GROUP BY z.Zone
    ORDER BY total_revenue DESC
    LIMIT 1
"""

df_q3 = bq_client.query(coursework_q3).to_dataframe()
pickup_zone = df_q3['pickup_zone'].iloc[0]
revenue = df_q3['total_revenue'].iloc[0]
print(f"Result: The pickup zone with the largest revenue was `{pickup_zone}` with ${revenue:,.2f}")

**Q4 Largest Tip: Drop-off Zone** 

Identify the drop-off zone with the highest single `tip_amount` for trips picked up in the zone `East Harlem North` during November 2025.

In [ ]:
coursework_q4 = f"""
    SELECT
        z2.Zone AS dropoff_zone,
        t.tip_amount AS largest_tip
    FROM `{FACT_TABLE}` t
    JOIN `{ZONES_TABLE}` z1 ON t.PULocationID = z1.LocationID
    JOIN `{ZONES_TABLE}` z2 ON t.DOLocationID = z2.LocationID
    WHERE z1.Zone = 'East Harlem North'
      AND t.tip_amount > 0
      AND t.lpep_pickup_datetime >= '2025-11-01'
      AND t.lpep_pickup_datetime <  '2025-12-01'
    ORDER BY largest_tip DESC
    LIMIT 1
"""

df_q4 = bq_client.query(coursework_q4).to_dataframe()
dropoff_zone = df_q4['dropoff_zone'].iloc[0]
largest_tip = df_q4['largest_tip'].iloc[0]

print(f"Result: The drop-off zone with the largest tip was `{dropoff_zone}` with ${largest_tip:,.2f}")

#### Gold Layer Materialization

In [ ]:
gold_sql = f"""
    CREATE OR REPLACE TABLE `{GOLD_TABLE}`
    PARTITION BY pickup_date
    AS
    SELECT
        DATE(t.lpep_pickup_datetime) AS pickup_date,
        z.Borough,
        z.Zone,
        SUM(t.total_amount) AS daily_revenue,
        SUM(t.trip_distance) AS total_distance,
        COUNT(*) AS trip_count
    FROM `{FACT_TABLE}` t
    JOIN `{ZONES_TABLE}` z ON t.PULocationID = z.LocationID
    GROUP BY 1, 2, 3
"""

try:
    bq_client.query(gold_sql).result()
    print(f"Gold table created.")
    print("This table is now optimized and partitioned by 'pickup_date'.")
except Exception as e:
    print(f"Gold layer creation failed: {e}")

#### Final Validation

In [ ]:
# quick preview of the gold layer
df_gold_preview = bq_client.query(f"SELECT * FROM `{GOLD_TABLE}` LIMIT 5").to_dataframe()
display(df_gold_preview)

#### Infrastructure Teardown

Remove transient staging tables (ext_green_taxi and zones) to maintain a clean environment and minimize metadata clutter, leaving only the optimized `gold layer` for reporting.

In [ ]:
# list of staging tables to decommission
staging_tables = [FACT_TABLE, ZONES_TABLE]

for table in staging_tables:
    bq_client.delete_table(table, not_found_ok=True)
    print(f"Decommissioned: {table}")

print("\nEnvironment Finalized. Staging assets removed.")